### Import of files

In [18]:
import json
import abc
import matplotlib.pyplot as plt
import numpy as np
import requests
from datetime import datetime, timedelta
from pytz import UTC
import pickle

In [19]:
apikeys = {'metObs': ""}

In [20]:
class weather_int(abc.ABC):
    @abc.abstractmethod
    def __init__(self, starttime: datetime, endtime: datetime) -> None:
        self._starttime = starttime
        self._endtime = endtime

    @property
    def endtime(self):
        return self._endtime.astimezone(UTC)
    
    @endtime.setter
    def endtime(self, date: datetime):
        self._endtime = date

    @property
    def starttime(self):
        return self._starttime.astimezone(UTC)
    
    @starttime.setter
    def starttime(self, date: datetime):
        self._starttime = date

    @abc.abstractmethod
    def output_csv(self, data, print_header = True, filename = "./weather.csv" ):
        pass

    @abc.abstractmethod
    def extract_weather(self):
        pass




class metObs(weather_int):
    _apikey = "api-key=" + apikeys["metObs"]
    _baselink = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items/?"
    def __init__(self, starttime: datetime, endtime: datetime, station = 6030) -> None:
        super().__init__(starttime, endtime)
        self._station = station  # Default Aalborg
        # Set parameter config
        self._parameter_id = {
            "humidity":[True, 10, "%"],
            "precip_dur_past10min":[True, 10, "kg/m²"],
            "precip_past10min":[True, 10, "min"],
            "precip_past1min":[True, 10, "min"],
            "pressure":[True, 10, "hPa"],
            "pressure_at_sea":[False, 10, "hPa"],
            "temp_dew":[True, 10, "degC"],
            "temp_dry":[True, 10, "degC"],
            "temp_grass":[False, 10, "degC"],
            "visib_mean_last10min":[False, 10, "m"],
            "visibility":[False, 10, "m"],
            "weather":[False, 10, "Code"],
            "wind_dir":[False, 10, "degree"],
            "wind_max":[False, 10, "m/s"],
            "wind_min":[False, 10, "m/s"],
            "wind_speed":[False, 10, "m/s"],
            "precip_dur_past1h":[False, 60, "min"],
            "precip_past1h":[False, 60, "kg/m²"],
            "temp_max_past1h":[False, 60, "degC"],
            "temp_mean_past1h":[False,60, "degC"],
            "temp_min_past1h":[False, 60, "degC"],
            "humidity_past1h":[False, 60, "%"],
            "temp_grass_max_past1h":[False,60,"degC"],
            "temp_grass_mean_past1h":[False,60,"degC"],
            "temp_grass_min_past1h":[False,60,"degC"],
            "wind_dir_past1h":[False,60,"degree"],
            "wind_gust_always_past1h":[False,60,"m/s"],
            "wind_max_per10min_past1h":[False,60,"m/s"],
            "wind_min_past1h":[False,60,"m/s"],
            "wind_speed_past1h":[False,60,"m/s"],
            }

    # Dataclass for processed JSON Query.    
    class metObsSample:
        def __init__(self, sample) -> None:
            properties = sample["properties"]
            try:
                self.coord = sample["geometry"]["coordinates"]
            except:
                self.coord = None
            self.observedTime = datetime.strptime(properties["observed"], "%Y-%m-%dT%H:%M:%S%z")
            self.parameterId = properties["parameterId"]
            self.value = properties["value"]
            pass 
    
    @property
    def station(self):
        if self._station / 10000 < 1:
            return "0" + str(self._station)
        else:
            return str(self.station)
    
    @station.setter
    def station(self, val:int):
        self._station = val

    @property
    def query_date(self):
        format_string = "%Y-%m-%dT%H:%M:%SZ"
        starttime = datetime.strftime(self.starttime - timedelta(minutes=self.starttime.minute, seconds=self.starttime.second), format_string)
        endtime = datetime.strftime(self.endtime - timedelta(minutes=self.endtime.minute, seconds=self.endtime.second), format_string)
        return f"{starttime}/{endtime}"

    @property
    def parameters(self):
        keys = list(self._parameter_id.keys())
        params = list(key for key in keys if self._parameter_id[key][0] == True)
        return params
    @parameters.setter
    def parameters(self, parameter): 
        self._parameter_id[parameter][0] = not self._parameter_id[parameter][0] 
        return

    # Request a parameter from DMI API.
    def _extract_parameter(self, parameter: str):
        count = int((self.endtime - self.starttime).total_seconds() // (self._parameter_id[parameter][1]*60))
        query = self._baselink + self._apikey + f"&stationId={self.station}&parameterId={parameter}&datetime={self.query_date}&limit={str(count+500)}"
        r = requests.request("GET", query)
        x = [self.metObsSample(index) for index in json.loads(r.content)["features"]]
        return x
    
    #Extract All active paramters from DMI.
    def extract_weather(self, sorttime = False):
        output = {}
        for param in self.parameters:
            tmp = self._extract_parameter(param)
            if len(tmp) == 0:
                    self.parameters = param # Remove from parameter list
                    continue
            if sorttime:
                for data in tmp:
                    if data.observedTime not in output:
                        output[data.observedTime] = {param:data.value}
                    else:
                        output[data.observedTime][param] = data.value
            else:
                output[param] = tmp
        if sorttime: output =  {date:output[date] for date in sorted(output)}
        return output
    
    # Format as CSV, and print to file 
    def output_csv(self, data, print_header=True, filename="./weather.csv"):
        def generate_line(line):
            output = line[0]
            for i in line[1:]:
                output = output + "," + str(i)
            return output
        format_str = "%Y-%m-%dT%H:%M:%SZ"
        params = self.parameters
        header1 = generate_line(["Time"] + params)
        header2 = generate_line(["UTC"] + list(self._parameter_id[x][2] for x in params))    
        with open(filename, 'w') as f:
            if print_header:
                f.write(header1 + "\n")
                f.write(header2 + "\n")
            for item in data:
                line = [datetime.strftime(item, format_str), *(0 for _ in range(len(params)))] 
                for i, param in enumerate(params):
                    if param in data[item]:
                        line[i+1] = data[item][param]
                    else:
                        line[i+1] = "NaN"
                f.write(generate_line(line) + "\n")
        return
    







### Open data files, and concatenate to a single arrays

In [21]:
years = [2014, 2016, 2018, 2020, 2022, 2024, 2025]

precip_1m_val = []
precip_1m_time = []
precip_10m_val = []
precip_10m_time = []
for i in range(1,len(years)):
    with open(f"dmi_{years[i-1]}_{years[i]}.bin", 'rb') as f:
        data = pickle.loads(f.read())
    f.close()
    precip_1m_tmp = data["precip_past1min"][:-1]
    precip_1m_val_tmp = np.flip(np.array([i.value for i in precip_1m_tmp]))
    precip_1m_time_tmp = np.flip(np.array([i.observedTime.timestamp() for i in precip_1m_tmp]))
    # precip_1m_time = precip_1m_time - precip_1m_time[0]
    precip_10m_tmp = data["precip_past10min"][:-1]
    precip_10m_val_tmp = np.flip(np.array([i.value for i in precip_10m_tmp]))
    precip_10m_time_tmp = np.flip(np.array([i.observedTime.timestamp() for i in precip_10m_tmp]))
    print(f"t: {precip_10m_tmp[0].observedTime}, {precip_10m_tmp[-1].observedTime}")
    print(f"10m: {precip_10m_time_tmp[0]}, {precip_10m_time_tmp[-1]}")
    print(f"1m: {precip_1m_time_tmp[0]}, {precip_1m_time_tmp[-1]}")
    precip_1m_time.extend(precip_1m_time_tmp)
    precip_1m_val.extend(precip_1m_val_tmp)
    precip_10m_time.extend(precip_10m_time_tmp)
    precip_10m_val.extend(precip_10m_val_tmp)


t: 2016-01-01 00:00:00+00:00, 2014-01-01 00:10:00+00:00
10m: 1388535000.0, 1451606400.0
1m: 1423023720.0, 1451571000.0
t: 2018-01-01 00:00:00+00:00, 2016-01-01 00:10:00+00:00
10m: 1451607000.0, 1514764800.0
1m: 1451641320.0, 1514733600.0
t: 2020-01-01 00:00:00+00:00, 2018-01-01 00:10:00+00:00
10m: 1514765400.0, 1577836800.0
1m: 1514780520.0, 1577727000.0
t: 2022-01-01 00:00:00+00:00, 2020-01-01 00:10:00+00:00
10m: 1577837400.0, 1640995200.0
1m: 1577867520.0, 1640946000.0
t: 2024-01-01 00:00:00+00:00, 2022-01-01 00:10:00+00:00
10m: 1640995800.0, 1704067200.0
1m: 1641045720.0, 1704066000.0
t: 2025-01-01 00:00:00+00:00, 2024-01-01 00:10:00+00:00
10m: 1704067800.0, 1735689600.0
1m: 1704067320.0, 1735682400.0


### Reconstruct the precip_1m_time
This is done by appending 0s for any minute, where data does not exist in 1m, as DMI does not push 1m, if it has not rained for 10 min.

In [ ]:
ran = np.arange(precip_10m_time[0], precip_10m_time[-1], 60)
val = []

idx = 0
for i, t in enumerate(ran):
    if idx < len(precip_1m_time) and (precip_1m_time[idx] == t):
        val.append(precip_1m_val[idx])
        idx += 1
    else:
        val.append(0.0)

val = np.array(val)

### Create running mean from convolve
Using a window size of 6

In [44]:
w = 6
rm_data = np.convolve(val,np.ones(w)/w, mode='valid')
rm_data = np.round(rm_data, 2)
print(np.unique(rm_data))
print(len(np.unique(rm_data)))

[0.   0.02 0.03 0.05 0.07 0.08 0.1  0.12 0.13 0.15 0.17 0.18 0.2  0.22
 0.23 0.25 0.27 0.28 0.3  0.32 0.33 0.35 0.37 0.38 0.4  0.42 0.43 0.45
 0.47 0.48 0.5  0.52 0.53 0.55 0.57 0.58 0.6  0.62 0.63 0.65 0.67 0.68
 0.7  0.72 0.73 0.75 0.77 0.78 0.8  0.82 0.83 0.85 0.87 0.9  0.92 0.93
 0.98 1.   1.02 1.03 1.05 1.07 1.08]
63


### Create Markov model from running mean

In [ ]:
from tqdm import tqdm
def get_state(state_values, curr_value):
    return np.where(np.isclose(state_values, curr_value))[0][0]

states_values = np.unique(rm_data)
print(states_values)
s_range = np.arange(len(states_values))
print(s_range)

state_counter = np.zeros((len(states_values),len(states_values)),dtype=np.float32)

prev_state = get_state(states_values, rm_data[0])
for i in tqdm(range(1, len(rm_data))):
    gcs = get_state(states_values, rm_data[i])
    state_counter[prev_state, gcs] += 1
    prev_state = gcs

P = state_counter

[0.   0.02 0.03 0.05 0.07 0.08 0.1  0.12 0.13 0.15 0.17 0.18 0.2  0.22
 0.23 0.25 0.27 0.28 0.3  0.32 0.33 0.35 0.37 0.38 0.4  0.42 0.43 0.45
 0.47 0.48 0.5  0.52 0.53 0.55 0.57 0.58 0.6  0.62 0.63 0.65 0.67 0.68
 0.7  0.72 0.73 0.75 0.77 0.78 0.8  0.82 0.83 0.85 0.87 0.9  0.92 0.93
 0.98 1.   1.02 1.03 1.05 1.07 1.08]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62]


100%|██████████| 5785904/5785904 [02:34<00:00, 37482.86it/s]


### Normalize Markov Model

In [46]:
P = state_counter
for i in range(len(states_values)):
    state_sum = np.sum(P[i,:])
    if state_sum > 0:
        P[i,:] = P[i,:] / state_sum
print(P)

[[9.9636424e-01 3.5178536e-03 8.8729241e-05 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.3371706e-01 7.8469175e-01 7.9621024e-02 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [5.6593097e-03 2.8534710e-01 5.6505847e-01 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  1.0000000e+00 0.0000000e+00]]


### Save Markov Model

In [ ]:
np.save('aau_ma_model',P)
np.save('aau_ma_states',states_values)

### Plot heatmap

In [48]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))
sns.heatmap(state_counter, 
            annot=True, 
            fmt=".1f", 
            cmap="viridis", 
            xticklabels=states_values,
            yticklabels=states_values,)
plt.xlabel("Next State")
plt.ylabel("Current State")
plt.title("Markov Transition Matrix Heatmap")
plt.tight_layout()
plt.show()

### Example of creating rain from the markov model

In [ ]:
N = len(val)
states = np.zeros(N, dtype=np.int8)
fake_rain = np.zeros(N, dtype=np.float32)
for i in tqdm(range(1, N)):
    prev_state = states[i-1]
    curr_state = np.random.choice(s_range, 1, True, P[prev_state,:])
    states[i] = curr_state[0]
    fake_rain[i] = states_values[curr_state]

  0%|          | 0/5785909 [00:00<?, ?it/s]/tmp/ipykernel_917637/975492797.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fake_rain[i] = states_values[curr_state]
 79%|███████▉  | 4581167/5785909 [02:11<00:36, 33274.04it/s]